# SnakeQL
# An SQL like Database built entirely within Python

In [2]:
#libraries imported
import json
import os
import threading
import datetime
from decimal import Decimal
import ply.lex as lex
import ply.yacc as yacc

In [3]:
class DatabaseError(Exception):
    pass

In [4]:
class TableExistsError(DatabaseError):
    pass

In [5]:
class ColumnNotFoundError(DatabaseError):
    pass

In [6]:
class DataTypeError(DatabaseError):
    pass

In [7]:
class TransactionError(DatabaseError):
    pass

In [8]:
class Transaction:
    def __init__(self, isolation_level='READ COMMITTED'):
        self.operations = []
        self.active = False
        self.isolation_level = isolation_level

In [ ]:
class Database:
    def __init__(self, db_path='db'):
        self.db_path = db_path
        if not os.path.exists(db_path):
            os.makedirs(db_path)
        self.tables = {}
        self.locks = {}
        self.transaction = Transaction()
        self.global_lock = threading.Lock()
        
    def begin_transaction(self, isolation_level='READ COMMITTED'):
        if self.transaction.active:
            raise TransactionError("Transaction already active")
        self.transaction.active = True
        self.transaction.operations = []
        self.transaction.isolation_level = isolation_level
        print(f"Transaction started with isolation level {isolation_level}")

    def commit(self):
        if not self.transaction.active:
            raise TransactionError("No active transaction")
        for op in self.transaction.operations:
            op()
        self.transaction.active = False
        self.transaction.operations = []
        print("Transaction committed")

    def rollback(self):
        if not self.transaction.active:
            raise TransactionError("No active transaction")
        self.transaction.active = False
        self.transaction.operations = []
        print("Transaction rolled back")

    def create_table(self, table_name, columns, index_columns=None):
        if table_name in self.tables:
            raise TableExistsError(f"Table '{table_name}' already exists")
        table = {
            'columns': columns,
            'rows': [],
            'indexes': {}
        }
        if index_columns:
            for col in index_columns:
                if col not in [c[0] for c in columns]:
                    raise ColumnNotFoundError(f"Index column '{col}' not in table columns")
                table['indexes'][col] = {}
        self.tables[table_name] = table
        self.locks[table_name] = threading.Lock()
        self._save_table(table_name)
        print(f"Table '{table_name}' created with columns {columns}")

    def drop_table(self, table_name):
        if table_name not in self.tables:
            raise TableNotFoundError(f"Table '{table_name}' does not exist")
        del self.tables[table_name]
        if os.path.exists(os.path.join(self.db_path, f"{table_name}.json")):
            os.remove(os.path.join(self.db_path, f"{table_name}.json"))
        print(f"Table '{table_name}' dropped.")

    def describe_table(self, table_name):
        table = self._get_table(table_name)
        print(f"Table '{table_name}' schema:")
        for col_name, col_type in table['columns']:
            print(f"- {col_name}: {col_type}")

    def alter_table_add(self, table_name, column_name, data_type):
        table = self._get_table(table_name)
        if column_name in [col[0] for col in table['columns']]:
            raise ColumnNotFoundError(f"Column '{column_name}' already exists in table '{table_name}'")
        table['columns'].append((column_name, data_type))
        for row in table['rows']:
            row[column_name] = None
        self._save_table(table_name)
        print(f"Table '{table_name}' altered. Column '{column_name}' added.")

    def alter_table_drop(self, table_name, column_name):
        table = self._get_table(table_name)
        if column_name not in [col[0] for col in table['columns']]:
            raise ColumnNotFoundError(f"Column '{column_name}' does not exist in table '{table_name}'")
        table['columns'] = [col for col in table['columns'] if col[0] != column_name]
        for row in table['rows']:
            if column_name in row:
                del row[column_name]
        self._save_table(table_name)
        print(f"Table '{table_name}' altered. Column '{column_name}' dropped.")

    def truncate_table(self, table_name):
        table = self._get_table(table_name)
        table['rows'] = []
        self._save_table(table_name)
        print(f"Table '{table_name}' truncated.")

    def insert_into(self, table_name, values):
        with self._get_lock(table_name):
            table = self._get_table(table_name)
            if len(values) != len(table['columns']):
                raise DatabaseError("Number of values does not match number of columns")
            row = {}
            for (col_name, col_type), value_str in zip(table['columns'], values):
                value = parse_value(value_str, col_type)
                row[col_name] = value
            self._update_indexes(table, row, add=True)
            if self.transaction.active:
                def op():
                    table['rows'].append(row)
                    self._save_table(table_name)
                self.transaction.operations.append(op)
            else:
                table['rows'].append(row)
                self._save_table(table_name)
            print(f"Inserted into '{table_name}': {row}")

    def select(self, table_name, columns=None, where=None, order_by=None, limit=None):
        with self._get_lock(table_name, shared=True):
            table = self._get_table(table_name)
            if columns is None or '*' in columns:
                columns = [col_name for col_name, col_type in table['columns']]
            result = []
            rows = table['rows']
            if where and isinstance(where, tuple) and where[0] == 'index':
                index_col, value = where[1], where[2]
                if index_col in table['indexes']:
                    index = table['indexes'][index_col]
                    rows = index.get(value, [])
                else:
                    pass
                where = None
            if where:
                rows = filter(where, rows)
            if order_by:
                reverse = False
                if order_by.startswith('-'):
                    reverse = True
                    order_by = order_by[1:]
                rows = sorted(rows, key=lambda x: x.get(order_by), reverse=reverse)
            if limit:
                rows = list(rows)[:limit]
            for row in rows:
                selected_row = {col: row.get(col) for col in columns}
                result.append(selected_row)
            print(f"Selected from '{table_name}':")
            for row in result:
                print(row)
            return result

    def update(self, table_name, set_values, where=None):
        with self._get_lock(table_name):
            table = self._get_table(table_name)
            updated_rows = 0
            for row in table['rows']:
                if where is None or where(row):
                    self._update_indexes(table, row, add=False)
                    for col, val_str in set_values.items():
                        col_type = self._get_column_type(table, col)
                        val = parse_value(val_str, col_type)
                        row[col] = val
                    self._update_indexes(table, row, add=True)
                    updated_rows += 1
            if self.transaction.active:
                def op():
                    self._save_table(table_name)
                self.transaction.operations.append(op)
            else:
                self._save_table(table_name)
            print(f"Updated {updated_rows} rows in '{table_name}'")

    def delete_from(self, table_name, where=None):
        with self._get_lock(table_name):
            table = self._get_table(table_name)
            new_rows = []
            deleted_rows = 0
            for row in table['rows']:
                if where is None or not where(row):
                    new_rows.append(row)
                else:
                    self._update_indexes(table, row, add=False)
                    deleted_rows += 1
            table['rows'] = new_rows
            if self.transaction.active:
                def op():
                    self._save_table(table_name)
                self.transaction.operations.append(op)
            else:
                self._save_table(table_name)
            print(f"Deleted {deleted_rows} rows from '{table_name}'")

    def _save_table(self, table_name):
        table = self.tables[table_name]
        table_data = {
            'columns': table['columns'],
            'rows': table['rows'],
            'indexes': {col: dict(index) for col, index in table.get('indexes', {}).items()}
        }
        with open(os.path.join(self.db_path, f"{table_name}.json"), 'w') as f:
            json.dump(table_data, f, default=str)

    def _load_table(self, table_name):
        try:
            with open(os.path.join(self.db_path, f"{table_name}.json"), 'r') as f:
                table_data = json.load(f)
                table_data['rows'] = [
                    {col: parse_loaded_value(val, col_type)
                     for (col, col_type), val in zip(table_data['columns'], row.values())}
                    for row in table_data['rows']
                ]
                table_data['indexes'] = {
                    col: {parse_loaded_value(key, self._get_column_type(table_data, col)): val
                          for key, val in index.items()}
                    for col, index in table_data.get('indexes', {}).items()
                }
                self.tables[table_name] = table_data
        except FileNotFoundError:
            raise TableNotFoundError(f"Table '{table_name}' does not exist")

    def _get_table(self, table_name):
        if table_name not in self.tables:
            self._load_table(table_name)
        return self.tables[table_name]

    def _get_lock(self, table_name, shared=False):
        if table_name not in self.locks:
            self.locks[table_name] = threading.Lock()
        lock = self.locks[table_name]
        return lock

    def _get_column_type(self, table, column_name):
        for col_name, col_type in table['columns']:
            if col_name == column_name:
                return col_type
        raise ColumnNotFoundError(f"Column '{column_name}' does not exist")

    def _update_indexes(self, table, row, add=True):
        for index_col, index in table.get('indexes', {}).items():
            key = row.get(index_col)
            if key is None:
                continue
            if add:
                if key not in index:
                    index[key] = []
                index[key].append(row)
            else:
                if key in index and row in index[key]:
                    index[key].remove(row)
                    if not index[key]:
                        del index[key]


In [9]:
def parse_value(value_str, data_type):
    value_str = value_str.strip()
    if data_type == 'int':
        return int(value_str)
    elif data_type == 'float':
        return float(value_str)
    elif data_type == 'str':
        if value_str.startswith('"') and value_str.endswith('"'):
            return value_str[1:-1]
        else:
            raise DataTypeError(f"Invalid string value: {value_str}")
    elif data_type == 'bool':
        if value_str.lower() in ('true', 'false'):
            return value_str.lower() == 'true'
        else:
            raise DataTypeError(f"Invalid boolean value: {value_str}")
    elif data_type == 'date':
        try:
            return datetime.datetime.strptime(value_str.strip('"'), '%Y-%m-%d').date()
        except ValueError:
            raise DataTypeError(f"Invalid date format: {value_str}")
    elif data_type == 'datetime':
        try:
            return datetime.datetime.strptime(value_str.strip('"'), '%Y-%m-%d %H:%M:%S')
        except ValueError:
            raise DataTypeError(f"Invalid datetime format: {value_str}")
    else:
        raise DataTypeError(f"Unsupported data type: {data_type}")

In [10]:
def parse_loaded_value(value_str, data_type):
    if data_type == 'int':
        return int(value_str)
    elif data_type == 'float':
        return float(value_str)
    elif data_type == 'str':
        return value_str
    elif data_type == 'bool':
        return value_str == 'True'
    elif data_type == 'date':
        return datetime.datetime.strptime(value_str, '%Y-%m-%d').date()
    elif data_type == 'datetime':
        return datetime.datetime.strptime(value_str, '%Y-%m-%d %H:%M:%S')
    else:
        return value_str
